In [1]:
# allows you to access the src module
import sys
sys.path.append("../")

from data_helper import *

# Data Cleaning & Preprocessing Techniques

This project is a simple walk-through of various data cleaning and preprocessing techniques to consider before doing data analysis.

### Why

Raw data often contains errors, missing values, inconsistencies, and other issues that can negatively impact the quality and reliability of any downstream analyses. By cleaning up, and preprocessing data, one can ensure that the data they are working with is accurate, complete, and in a format that is appropriate for the intended analysis or modeling task. This can lead to more reliable and accurate results, as well as more efficient use of time and resources.

> **Note**: Data cleaning and preprocessing is an iterative process as new issues can be uncovered during analysis or modeling.

### What this IS NOT!

Data cleaning and preprocessing is not a way for you to introduce your own bias. This can happen unintentionally, but the techniques used should be conducted carefully to avoid overemphasizing or underemphasizing aspects of your data.

For example, if a dataset contains missing values, one common approach to address this is to impute the missing values. However, if the missing values are not random, but instead systematically related to other variables in the dataset, then the imputed values may introduce bias into the dataset.

### Data Cleaning (DC)
**Data Cleaning** involves Identifying and correcting errors and inconsistencies in the data such as:

- incorrect formatting
- missing values
- duplicates
- outliers

The goal of data cleaning is to ensure that the data is **accurate**, **complete**, and **consistent**.

### Data Preprocessing (DP)
Data preprocessing on the otherhand, involves transforming the data into a format that is more suitable for analysis or modeling. Data preprocessing techniques may include:

- transforming the data into a different representation
- scaling or standardizing variables
- encoding categorical variables 
- reducing the dimensionality of the data

## DC Techniques: Incorrect Formatting
Incorrect formatting refers to data that fails to meet the expected or required format for a particular type of data. For example, if you have a dataset of dates and some of the dates are in the format "dd/mm/yyyy" while others are in the format "mm/dd/yyyy," that would be considered incorrect formatting. Additionally, if you have a dataset of genders and some of those values resolve to "Man" while others "Male", this would also be considered incorrect formatting. There are various types of incorrect formatting. Let us go over the following:

- Column Headers

- Inconsistent Values

- Inconsistent Types

- Text Casing

### Column Headers
This refers to datasets with column headers that differ in standardization. Consider the following cell:

In [2]:
users = generate_users()
users.head(2)

,ID,name,AgE,dateOfBirth,current_COUNTRY
0,645ba30d2ae4594cf0e635f4,Amy Martin,55,1964-09-06,Albania
1,645ba30d2ae4594cf0e635f5,Amy Smith,64,1964-08-13,Kuwait


Notice how the columns in the above dataset don't adhere to the same conventions. This isn't ideal. Let's clean this up.

In [3]:
users.columns = ["id", "name", "age", "date_of_birth", "current_country"]
users.head(2)

,id,name,age,date_of_birth,current_country
0,645ba30d2ae4594cf0e635f4,Amy Martin,55,1964-09-06,Albania
1,645ba30d2ae4594cf0e635f5,Amy Smith,64,1964-08-13,Kuwait


Yes! Notice how all of the columns follow the same convention. That is:

- lowercase
- snakecase

This is much cleaner and adheres to python naming conventions which will make data analysis much simpler!

### Inconsistent Values
Inconsistent values refers to a field, containing different values that are trying to represent the same thing. For example:

In [4]:
states = generate_states()
states.state.value_counts()

Ohio        17
OH          17
Texas       16
TX          16
AL          13
Alabama     13
MI           4
Michigan     4
Name: state, dtype: int64

From real world knowledge, we know that `Alabama` and `AL` are refering to the same state. This will cause confusion and added complexity when we get to conducting an analysis. Let's clean this up!

In [5]:
state_map = dict(zip(["Ohio", "Michigan", "Alabama", "Texas"], ["OH", "MI", "AL", "TX"]))
state_mask = states.state.map(state_map)
states.loc[states.state[state_mask.notna()].index, "state"] = state_mask
states.state.value_counts()

OH    34
TX    32
AL    26
MI     8
Name: state, dtype: int64

Great! Now every state adheres to it's abbreviation. Making the data more consistent!

### Inconsistent Types
Inconsistent types refers to data types in a column that don't match up. For example:

In [6]:
vals = pd.DataFrame([True, "true", 1], columns=["is_true"])
vals

,is_true
0,True
1,true
2,1


Notice the above values range from Booleans, strings, to ints. We need to make the types for this consistent. Let's get started!

In [7]:
vals.is_true = vals.is_true.apply(lambda x: bool(x))
vals

,is_true
0,True
1,True
2,True


Great! Now all of the values match the same type: `Boolean`!

### Text Casing
Text casing can be applied to the dataframe and to a specific row. This means values vary in casing and forcing them to adhere to the same sensitivity will make analysis all the more merry!

In [8]:
genders = generate_gender()
genders.value_counts()

gender
MAN       24
man       17
DND       16
dnd       15
WOMAN     14
woman     14
dtype: int64

Take a look, we have the same value producing different stats due to the casing of them. Let's fix this!

In [9]:
genders.gender = genders.gender.str.lower()
genders.value_counts()

gender
man       41
dnd       31
woman     28
dtype: int64

Now we have more accurate stats!

## DC Techniques: Missing Values

Missing values refer to data that is missing from specific rows or values. There are a variety of reasons for this, but nonetheless, handling them is, or can be, important! Missing value techniques include:

- Drop Values

- Imputing Values

- Do Nothing!

### Drop Values
The simplest route to take when figuring out how to handle missing values is to drop them. Drop anything in our data that isn't producing value or analytical insights. Let us explore!

In [10]:
users_missing = generate_users_missing()
display(users_missing.info(show_counts=True))
display(users_missing.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               19 non-null     float64
 1   name             21 non-null     object 
 2   date_of_birth    22 non-null     object 
 3   current_country  20 non-null     object 
dtypes: float64(1), object(3)
memory usage: 928.0+ bytes


None

,ID,name,date_of_birth,current_country
0,1.0,mtpty,None,Mexico
1,4.0,iqoqb,1980-06-03,Mexico
2,NaN,vjltw,1997-05-25,None
3,NaN,xxnox,1965-10-02,None
4,13.0,bdssu,2020-12-28,None


Say we want to calculate the age of each user, but from the above, we notice that there certain users didn't enter their birthday. Let's go ahead and drop them so we can execute our calculation only on users who've entered their birthday.

In [11]:
users_missing_clean = users_missing.dropna(subset=["date_of_birth"]).reset_index(drop=True)
display(users_missing_clean.info(show_counts=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               16 non-null     float64
 1   name             18 non-null     object 
 2   date_of_birth    22 non-null     object 
 3   current_country  17 non-null     object 
dtypes: float64(1), object(3)
memory usage: 832.0+ bytes


None

Great! Notice how there are now the same number of `Non-Null Count` users with `date_of_birth` as there are `entries`. This means we've successfully dropped users without a `date_of_birth`!

### Impute Values
Another way to handle missing values is to impute them. This means we estimate what the missing value could be. Imputing is great when you need data to run your analysis or when you have to estimate something.

> **Note**: The imputation methods shown can be very effective, but also very elementary. There are other, more advanced statistical methods to impute data such as: regression imputation, bayesian imputation, and more.

#### Mean
Mean imputation takes the average of the values in that column and appends that to the missing rows. For example:

In [12]:
age_missing = generate_age_missing(100)
print("Missing Values: ", age_missing.age.isna().sum())
display(age_missing.head())

Missing Values:  18


,age
0,19.0
1,42.0
2,71.0
3,66.0
4,49.0


See, where missing ages! Let's go ahead and do a mean imputation!

In [13]:
mean_age = round(age_missing.mean()).age
print("Mean Age to Impute:", mean_age)

age_missing.loc[age_missing.age[age_missing.age.isna()].index, "age"] = mean_age
print("Missing Values: ", age_missing.age.isna().sum())

Mean Age to Impute: 51.0
Missing Values:  0


#### Median

Median imputation refers to imputing missing values based on the middle value of the dataset that has been ordered from smallest to largest.

#### Mode
Mode imputation refers to imputing missing values based on the value that shows up the most frequent in a given dataset.

## DC Techniques: Duplicates

## DC Techniques: Outliers

## DP Techniques: Transforming Representation

## The Remaining DP Techniques

- scaling or standardizing variables
- encoding categorical variables 
- reducing the dimensionality of the data